# Ejercicio 3: Preprocesamiento

## Objetivo de la práctica

1. Comprender y aplicar normalización, tokenización, stopwords, stemming y n-gramas.
2. Medir el impacto de cada paso en el vocabulario y los tokens.

#### 0. Cargar el Corpus

Vamos a trabajar con el corpus de Movie Reviews de IMDB

In [14]:
import pandas as pd

In [15]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


#### 1. Limpieza 

Limpiar los documentos de caracteres que no corresponden

In [16]:
doc = df.iloc[0]['review']
doc

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [17]:
doc.replace('<br />', '').replace('>', '').replace('/', '')

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [18]:
import re

In [19]:
def clean_text(doc):
    return re.sub(pattern=r'<.*?>', repl='', string=doc).replace('.', ' ').replace(',', '').replace('(', '').replace(')', '').replace('"', '').replace("'", '').replace("\x08", '')

In [20]:
df['review_clean'] = df['review'].apply(clean_text)

#### 2. Normalización

Convertir todos los tokens a minúsculas.

Elimina puntuación y símbolos no alfabéticos.

In [21]:
# Convertir a minúsculas
df['review_clean'] = df['review_clean'].str.lower()

In [22]:
# Tokenización simple
df['tokens'] = df['review_clean'].apply(lambda x: x.split())

#### 3. Eliminación de Stopwords

Eliminar las palabras vacías (stopwords) usando una lista estándar de la librería _nltk_.

In [23]:
import nltk
from nltk.corpus import stopwords

# Cargar stopwords
nltk.data.path.append('/kaggle/input/nltk-stopwords')
stop_words = set(stopwords.words('english'))

# Función para eliminar stopwords
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

df['tokens_no_stop'] = df['tokens'].apply(remove_stopwords)

#### 4. Stemming 

Reducir el espacio de palabras

In [24]:
from nltk.stem import porter

stemmer = porter.PorterStemmer()

# Función para aplicar stemming
def apply_stemming(tokens):
    return [stemmer.stem(token) for token in tokens]

df['tokens_stemmed'] = df['tokens_no_stop'].apply(apply_stemming)

#### 5. Verificar la diferencia

Comparar el tamaño del diccionario de términos del corpus antes y después de aplicar el preprocesamiento 

In [25]:
# Vocabulario original (después de limpieza)
vocab_original = set()
for tokens in df['tokens']:
    vocab_original.update(tokens)

# Vocabulario después de preprocesamiento completo
vocab_processed = set()
for tokens in df['tokens_stemmed']:
    vocab_processed.update(tokens)

print(f"Vocabulario original: {len(vocab_original)} palabras")
print(f"Vocabulario procesado: {len(vocab_processed)} palabras")
print(f"Reducción: {len(vocab_original) - len(vocab_processed)} palabras")
print(f"Porcentaje reducido: {((len(vocab_original) - len(vocab_processed)) / len(vocab_original)) * 100:.2f}%")

Vocabulario original: 199622 palabras
Vocabulario procesado: 161460 palabras
Reducción: 38162 palabras
Porcentaje reducido: 19.12%
